# Expiscor 
This Notebook is a an exploration to Expiscor modules. 

The first module is `mpt.py`, which calculates the necessary functions Modern Portfolio Theory. 

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import requests
import plotly.express as px
import datetime

In [8]:
# Define a search query
def search(query):
  return pd.DataFrame(requests.get(f"http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={query}&region=1&lang=en").json()['ResultSet']['Result'])

In [47]:
# Example
search("S&P 500")

symbol                                     name exch type  exchDisp typeDisp
0   SQSP                        Squarespace, Inc.  NYS    S      NYSE   Equity
1   SMWB                          SIMILARWEB LTD.  NYS    S      NYSE   Equity
2  SHQAU  Shelter Acquisition Corporation I Units  NAS    S    NASDAQ   Equity
3   SKLZ                              Skillz Inc.  NYQ    S      NYSE   Equity
4    SPY                   SPDR S&P 500 ETF Trust  PCX    E  NYSEArca      ETF
5   SPCE           Virgin Galactic Holdings, Inc.  NYQ    S      NYSE   Equity
6   SRNE              Sorrento Therapeutics, Inc.  NMS    S    NASDAQ   Equity
7   SNDL                     Sundial Growers Inc.  NMS    S    NASDAQ   Equity
8     SQ                             Square, Inc.  NYQ    S      NYSE   Equity
9   SHOP                             Shopify Inc.  NYQ    S      NYSE   Equity

In [15]:
get_symbol("^SPX")

'S&P 500 INDEX'

In [13]:
def get_symbol(symbol):
    url = f"http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={symbol}&region=1&lang=en"

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

In [32]:
def validate(date_text):
  '''
  A function that validates if date_text is actually date_text
  '''
  try: 
      datetime.datetime.strptime(date_text, '%Y-%m-%d')
  except ValueError:
      raise ValueError("錯誤數據格式，必須是YYYY-MM-DD的日期格式")
def getport(start, end, symbols):
  '''
  A function that collects all data from a list of assets from Yahoo! Finance
  '''
  assets = [get_symbol(x) for x in symbols]
  validate(start)
  validate(end)
  # Define date_range
  date_range = pd.date_range(
    start = start,
    end = end,  
    freq = 'B', 
    normalize = True
    )
  
  
  df_list = [yf.Ticker(symbol).history(start=start, end=end) for symbol in symbols]
  for df, asset in zip(df_list, assets):
    col_list = df.drop('Close', axis = 1).columns
    df.drop(col_list, axis=1, inplace=True)
    df.rename(columns={'Close': asset}, inplace=True)
  df = pd.concat(df_list, axis=1, ignore_index=False)
  df = df.fillna(method='ffill').reindex(index=date_range, method='ffill')
  df.dropna(how='any', inplace=True)
  return df

In [51]:
get_symbol("^GSPC")

'S&P 500'

In [65]:
test = getport(start="2020-01-01",end="2021-01-01",symbols=["^GSPC","^IXIC"])

In [69]:
test

S&P 500  NASDAQ Composite
2020-01-02  3257.850098       9092.190430
2020-01-03  3234.850098       9020.769531
2020-01-06  3246.280029       9071.469727
2020-01-07  3237.179932       9068.580078
2020-01-08  3253.050049       9129.240234
...                 ...               ...
2020-12-28  3735.360107      12899.419922
2020-12-29  3727.040039      12850.219727
2020-12-30  3732.040039      12870.000000
2020-12-31  3756.070068      12888.280273
2021-01-01  3756.070068      12888.280273

[262 rows x 2 columns]

In [72]:
(test.iloc[-1]-test.iloc[0])/test.iloc[0]

S&P 500             0.152929
NASDAQ Composite    0.417511
dtype: float64

# Efficient Frontier Formula
Expected Return of the portfolio is the weighted sum of Expected Return of all assets
$$E(Rp)=\sum_{i} w_i E(R_i)$$
Expected Variance of the portfolio is the 
$$Var(R_p) = w^2_1Var(R_1)+w^2_2Var(R_2)+2w_1w_2$$


In [75]:
test.pct_change().cov()

S&P 500  NASDAQ Composite
S&P 500           0.000456          0.000451
NASDAQ Composite  0.000451          0.000484